In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)
import tensorflow as tf

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [57]:
mnist.test.images

0.0

In [15]:
# 128장의 이미지를 랜덤하게 공급
batch_x, batch_y = mnist.train.next_batch(128)
# 128x784 => 128(가로)x784(세로)? 이미지가 일렬로 들어왔다.(flatten)
print(batch_x.shape)

learning_rate = 0.1 # 학습률?
num_steps = 500 # epoch
batch_size = 128 # mini batch 애유 : 속도를 높이고, 지역해 문제 극복
display_step = 100

(128, 784)


In [16]:
# 은닉층 2개(층)
# 784x256 -> 256x256 -> 256x10
n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784
num_classes = 10

X = tf.placeholder('float', [None, num_input])# 128x784
Y = tf.placeholder('float', [None, num_classes]) # 128x10

In [17]:
# 가중치 : 설명렬, 계수
weights = {
    # 784x256
    'h1':tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    # 256x256
    'h2':tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    # 256x10
    'out':tf.Variable(tf.random_normal([n_hidden_2, num_classes])),
}
# h1 : 128x256으로 축소되서 w(가중치)를 곱한다.

# bias : activation
biases = {
    # 256, 변수 개수 만큼
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    # 256
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    # 10
    'out':tf.Variable(tf.random_normal([num_classes])),
}
# b1 : 256으로 줄어들었으니 바이어스를 256개 더해준다.

In [18]:
def neural_net(x):
    # 128x784, 784x256 => 128x256 전달 데이터 (tensor)
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # 128x256, 256x256 => 128x256
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # 128x256, 256x10
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer

In [35]:
logits =neural_net(X)

# softmax 확률값으로 계산
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

# 최적화기 : 기울기,  학습률
# adagrad : learning_rate 처음엔 크게 -> 점점 작게 끝나지않아도 0으로 만든다.
# momentum + propgrad(): adaptive 적응적으로 현재 상태를 보고?
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# 가장큰 값의 인덱스를 구함(one-hot-encoding) => target
# true, false => 0,1로 나온다.
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))

# 형변환
# 합을 평균함
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()


In [42]:
with tf.Session() as sess: # GPU 네트워크 연결 => 일반화
    sess.run(init)
    for step in range(1, num_steps+1): # epoch
        # 128x784, 128x10
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if step % display_step ==0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})
            print('Step'+str(step)+'Minibatch Loss  : {:.4f}'.format(loss)+', Training Accuracy : {:.3f}'.format(acc))
    print('테스트 정확도 : \n',sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

Step1Minibatch Loss  : 14528.2969, Training Accuracy : 0.297
Step100Minibatch Loss  : 249.7840, Training Accuracy : 0.898
Step200Minibatch Loss  : 96.1696, Training Accuracy : 0.875
Step300Minibatch Loss  : 42.6638, Training Accuracy : 0.914
Step400Minibatch Loss  : 60.6311, Training Accuracy : 0.875
Step500Minibatch Loss  : 90.3050, Training Accuracy : 0.844
테스트 정확도 : 
 0.8604


<tf.Tensor 'Equal_1:0' shape=(?,) dtype=bool>